# Agenda

- Power Meter
    - 何謂power meter
    - 來看看Power Meter的點表
    - Live Demo
    
- MQTT - 即時性應用
    - Mosquitto
    - paho
    
- thingspeak

----


## 在進入主題之前
----
### modbus格式介紹 

[參考](http://gridconnect.com/blog/tag/modbus-rtu/)

- Modbus/RTU: [start time] [Address 8bits + Function 8bits + Data Nx8bits + CRC 16bits] [End time]

- Modbus/TCP: [header 6byte + Address 8bits + Function 8bits + Data Nx8bits]

- Modbus/ASCII: 現在比較少人用，跳過不講

![格式](image/MODBUS-Frame.png)
        


### modbus常被採用的原因
        
- 公開發表並且無版稅要求

- 相對容易的工業網路部署

- 協定格式簡單，極省資源


----

## Modbus套件

----



### 網路上較多人提到的三個套件

[performance比較](https://stackoverflow.com/questions/17081442/python-modbus-library)

- modbus-tk: Modbus/RTU, Modbus/TCP

- pymodbus: 據說實作最完整，但使用資源相對的多，相依套件也多

- MinimalModbus: Modbus/RTU, Modbus/ASCII



### Modbus-tk

個人推薦使用

- 安裝方式
    - pip install serial
    - pip install modbus_tk

- 操作
    - [Python與PLC共舞](https://github.com/maloyang/PLC-Python)
     

----

## Power Meter

電力監控常見的元素

----



### Power Meter的點表

![Power Meter的點表](image/power_meter.gif)



### Power Meter的浮點數表示方式

![浮點數](image/power_meter_float.gif)

----
![float](image/mb_float.png)
- [Live Demo](Modbus.ipynb)


In [1]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import modbus_tk.modbus_tcp as modbus_tcp
import time
from struct import *

In [2]:
# Modbus/TCP Client Start

mbTimeout = 1
mb_ip = '209.97.161.199'
mb_port = 10502

master = modbus_tcp.TcpMaster(mb_ip, mb_port)
master.set_timeout(mbTimeout)

mbId = 1
addr = 0
try:
    # FC3
    rr = master.execute(mbId, cst.HOLDING_REGISTERS, addr, 3)
    print('read value:', rr)

except Exception as e:
    print("modbus test Error: " + str(e))

master._do_close()


read value: (112, 110, 109)


True

In [3]:
# if float

# Modbus/TCP Client Start

mbTimeout = 1
mb_ip = '209.97.161.199'
mb_port = 10502

master = modbus_tcp.TcpMaster(mb_ip, mb_port)
master.set_timeout(mbTimeout)

mbId = 1
addr = 20
try:
    # FC3
    rr = master.execute(mbId, cst.HOLDING_REGISTERS, addr, 2)
    print('read value:', rr)

    # convert to float:
    # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
    try:
        v_a_hi = rr[1]
        v_a_lo = rr[0]
        v_a = unpack('>f', pack('>HH', v_a_hi, v_a_lo))
        print('v_a=', v_a)
        #struct.unpack(">f",'\x42\xd8\x6b\x8d')
    except Exception as e:
        print(e)
    
except Exception as e:
    print("modbus test Error: " + str(e))

master._do_close()


read value: (39322, 16417)
v_a= (2.5250000953674316,)


True

In [ ]:
# TODO: 如果address 20開始有三個float表示電流值，要怎麼處理?


----

## MQTT - 即時性應用

已經可以採集資料了，來談談怎麼上傳雲端吧

----


### Mosquitto

一個broker套件，當然也可以做為client使用

- 以NB X260來說，4000多個連結沒有問題

![Mosquitto](image/Eclipse-Mosquitto-logo.png)


### paho

便利的MQTT client端套件

- [link](https://pypi.org/project/paho-mqtt/)
- install: `pip install paho-mqtt`

![paho](image/mqtt-paho-featured-image.jpg)


In [17]:
import paho.mqtt.client as mqtt
import time, json, datetime, sys, os

#==================
#== MQTT Functions

# 當地端程式連線伺服器得到回應時，要做的動作
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    # 將訂閱主題寫在on_connet中
    # 如果我們失去連線或重新連線時 
    # 地端程式將會重新訂閱
    client.subscribe('KH20221202_IoT_Data_Science/#')
    

# 當接收到從伺服器發送的訊息時要進行的動作
def on_message(client, userdata, msg):
    # 轉換編碼utf-8才看得懂中文
    try:
        topic = msg.topic
        value = msg.payload.decode('utf-8')
        dtime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print('(topic, value)=(%s, %s) @%s' %(topic, value, dtime))
    except Exception as e:
        print('read data exception: ', str(e))

def run_mqtt():
    try:
        # 連線設定
        # 初始化地端程式
        client = mqtt.Client()

        # 設定連線的動作
        client.on_connect = on_connect

        # 設定接收訊息的動作
        client.on_message = on_message

        # 設定登入帳號密碼
        #client.username_pw_set("try","xxxx")

        # 設定連線資訊(IP, Port, 連線時間)
        client.connect("broker.hivemq.com", 1883, 60)

        # 開始連線，執行設定的動作和處理重新連線問題
        # 也可以手動使用其他loop函式來進行連接
        #client.loop_forever()
        client.loop_start()    #start the loop

        time.sleep(30)

        client.disconnect() # disconnect gracefully
        client.loop_stop() # stops network loop
        
    except Exception as e:
        print('run mqtt error: ', str(e))

run_mqtt()


Connected with result code 0
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a1, 0.411) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a2, 0.403) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a3, 0.037) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v1, 238.586) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v2, 237.920) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v3, 238.479) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm2/a2, 2.801) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm2/total_e, 18684.180) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm2/v1, 238.506) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm2/v2, 241.255) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm2/v3, 240.098) @2022-12-24 01:19:05
(topic, value)=(KH20221202_IoT_Data_Science/farm

----

## Web API

適合非即時性的應用

----


----

### 運用 ThingSpeak收集資料

- https://thingspeak.com/
- 註冊登入後，於`https://thingspeak.com/channels`可以看到你的管理畫面
- [文件](https://indico.cern.ch/event/654636/contributions/2834242/attachments/1636045/2610243/MATLAB_IoT_public.pdf)
----

In [8]:
# continue push data

my_h = [65.46, 65.75, 62.49, 65.4, 65.89, 63.11]
my_t = [25.05, 24.72, 25.06, 24.66, 24.99, 24.58]

In [9]:
len(my_t)

6

In [10]:
import requests
import time

for i in range(len(my_t)):
    
    print('---- %s ----' %(i))
    print('H=%s %%, T=%s degree' %(my_h[i], my_t[i]))
    
    # push data
    # field1: T
    # field2: H
    url = 'https://api.thingspeak.com/update'
    api_key = '5IRM6UNIDXLCAPM1'
    field1 = my_t[i]
    field2 = my_h[i]
    data = {'api_key': api_key, 'field1':field1, 'field2':field2}
    r = requests.get(url, params=data)
    print(r)
    
    time.sleep(15)
    

---- 0 ----
H=65.46 %, T=25.05 degree
<Response [200]>
---- 1 ----
H=65.75 %, T=24.72 degree
<Response [200]>
---- 2 ----
H=62.49 %, T=25.06 degree
<Response [200]>
---- 3 ----
H=65.4 %, T=24.66 degree
<Response [200]>
---- 4 ----
H=65.89 %, T=24.99 degree
<Response [200]>
---- 5 ----
H=63.11 %, T=24.58 degree
<Response [200]>
